<a href="https://colab.research.google.com/github/SiaPorgaste/DATN/blob/main/weather_predict_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Change directory to your project folder
import os
os.chdir('/content/drive/MyDrive/Luan_van/LSTM')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns #To help plot
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler  #To scale data
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score #To evaluate model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model #To load model

# **Processing Data**

In [25]:
# Load the data
data = pd.read_csv('weather.csv')
data = data[data['province'] == 'Ho Chi Minh City']
data = data.dropna()
required_cols = ['max', 'min', 'wind', 'rain', 'humidi', 'cloud', 'pressure']
data.index = data.date
data = data.drop('date', axis=1)
data = data[required_cols].astype(float)
data.head()

,max,min,wind,rain,humidi,cloud,pressure
date,,,,,,,
1/1/2009,29.0,22.0,6.0,18.5,89.0,68.0,1010.0
1/1/2010,34.0,24.0,5.0,0.0,59.0,20.0,1010.0
1/1/2011,29.0,23.0,4.0,0.2,81.0,42.0,1008.0
1/1/2012,32.0,23.0,7.0,1.8,76.0,35.0,1012.0
1/1/2013,33.0,24.0,4.0,0.0,69.0,15.0,1010.0


In [43]:
# Normalize the Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

In [47]:
scaled_data[0]

array([0.36842105, 0.42857143, 0.16666667, 0.07303593, 0.84615385,
       0.68686869, 0.53333333])

# **QoL Function**

In [27]:
# Create Sequences Of Pressure
def create_sequences_custom(data, seq_length, col):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length][col])
    return np.array(X), np.array(y)

# **Get Model**

In [28]:
best_model = tf.keras.models.load_model('best_model.h5')
best_model_min = tf.keras.models.load_model('best_model_min.h5')
best_model_wind = tf.keras.models.load_model('best_model_wind.h5')
best_model_rain = tf.keras.models.load_model('best_model_rain.h5')
best_model_humi = tf.keras.models.load_model('best_model_humi.h5')
best_model_cloud = tf.keras.models.load_model('best_model_cloud.h5')
best_model_pressure = tf.keras.models.load_model('best_model_pressure.h5')

# **Get Time**

In [29]:
from datetime import datetime
import pandas as pd

FUTURE_DAYS = 30

# Get the current date
current_date = pd.Timestamp(datetime.now().date())
last_date = data.index[-1]
last_date = pd.to_datetime(last_date)
print(last_date)

2020-12-19 00:00:00


In [30]:
print(type(last_date))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [31]:
# Calculate the number of days between the last date and the current date
SEQ_LENGTH = (current_date - last_date).days

# Add 30 days
SEQ_LENGTH += FUTURE_DAYS

# Generate future dates for the next SEQ_LENGTH days
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=SEQ_LENGTH, freq='D')
future_dates_string = future_dates.strftime('%-m/%-d/%Y')

# Create a DataFrame with the future dates
future_dates_df = pd.DataFrame({'date': future_dates_string})

print(future_dates_df)

            date
0     12/20/2020
1     12/21/2020
2     12/22/2020
3     12/23/2020
4     12/24/2020
...          ...
1253   5/26/2024
1254   5/27/2024
1255   5/28/2024
1256   5/29/2024
1257   5/30/2024

[1258 rows x 1 columns]


# **Improved QoL of sequence creation**

In [32]:
# Create Sequences Of Pressure
def create_sequences_custom(data, seq_length, col):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length][col])
    return np.array(X), np.array(y)

# **Predict**

In [63]:
def predict_max_temp(SEQ_LENGTH, scaled_data=scaled_data, best_model=best_model):
    # Create sequences for the entire dataset
    X, y = create_sequences_custom(scaled_data, SEQ_LENGTH, 0)
    # Predict temperatures
    print("Predicting max temperature...")
    future_predictions = best_model.predict(X[-SEQ_LENGTH:])
    future_predictions_copies = np.repeat(future_predictions, 7, axis=-1)
    future_predicted_temp = scaler.inverse_transform(future_predictions_copies)[:,0]
    return future_predicted_temp

In [62]:
predict_max_temp(14)

Predicting max temperature...
1/1 [==============================] - 0s 37ms/step
[[0.5733602 ]
 [0.6037749 ]
 [0.5609629 ]
 [0.57563394]
 [0.5787264 ]
 [0.57532895]
 [0.6333331 ]
 [0.5477733 ]
 [0.54753345]
 [0.53347564]
 [0.51622605]
 [0.5370372 ]
 [0.60716015]
 [0.6003993 ]]


array([32.893845, 33.47172 , 32.658295, 32.937046, 32.9958  , 32.93125 ,
       34.03333 , 32.407692, 32.403137, 32.136036, 31.808294, 32.20371 ,
       33.536045, 33.40759 ], dtype=float32)

In [34]:
def predict_min_temp(SEQ_LENGTH, scaled_data=scaled_data, best_model_min=best_model_min):
    # Create sequences for the entire dataset
    X, y = create_sequences_custom(scaled_data, SEQ_LENGTH, 1)
    # Predict min temperatures
    print("Predicting min temperature...")
    future_predictions_min = best_model_min.predict(X[-SEQ_LENGTH:])
    future_predictions_min_copies = np.repeat(future_predictions_min, 7, axis=-1)
    future_predicted_min_temp = scaler.inverse_transform(future_predictions_min_copies)[:,1]
    return future_predicted_min_temp

In [64]:
predict_min_temp(14)

Predicting min temperature...
1/1 [==============================] - 2s 2s/step


array([23.709984, 24.33029 , 23.418762, 23.998804, 23.726255, 23.63004 ,
       24.780445, 23.075487, 23.58843 , 23.1191  , 22.919676, 23.384274,
       24.450891, 23.979239], dtype=float32)

In [35]:
def predict_wind(SEQ_LENGTH, scaled_data=scaled_data, best_model_wind=best_model_wind):
    # Create sequences for the entire dataset
    X, y = create_sequences_custom(scaled_data, SEQ_LENGTH, 2)
    # Predict wind spd
    print("Predicting wind speed...")
    future_predictions_wind = best_model_wind.predict(X[-SEQ_LENGTH:])
    future_predictions_wind_copies = np.repeat(future_predictions_wind, 7, axis=-1)
    future_predicted_wind = scaler.inverse_transform(future_predictions_wind_copies)[:,2]
    return future_predicted_wind

In [65]:
predict_wind(14)

Predicting wind speed...
1/1 [==============================] - 4s 4s/step


array([15.007101, 15.974679, 14.487368, 15.515438, 14.820433, 14.586952,
       16.71778 , 13.907597, 14.697321, 13.831434, 13.507408, 14.311316,
       16.09815 , 15.347006], dtype=float32)

In [36]:
def predict_rain(SEQ_LENGTH, scaled_data=scaled_data, best_model_rain=best_model_rain):
    # Create sequences for the entire dataset
    X, y = create_sequences_custom(scaled_data, SEQ_LENGTH, 3)
    # Predict rain
    print("Predicting rain...")
    future_predictions_rain = best_model_rain.predict(X[-SEQ_LENGTH:])
    future_predictions_rain_copies = np.repeat(future_predictions_rain, 7, axis=-1)
    future_predicted_rain = scaler.inverse_transform(future_predictions_rain_copies)[:,3]
    return future_predicted_rain

In [66]:
predict_rain(14)

Predicting rain...
1/1 [==============================] - 3s 3s/step


array([ 1.0019957, -1.7402431,  2.1995313, -1.3722593, -0.5969806,
       -0.4083555, -4.0016756,  3.1578243, -0.9279256,  0.5277531,
        1.958811 , -2.5963142, -4.220149 , -0.9629979], dtype=float32)

In [37]:
def predict_humi(SEQ_LENGTH, scaled_data=scaled_data, best_model_humi=best_model_humi):
    # Create sequences for the entire dataset
    X, y = create_sequences_custom(scaled_data, SEQ_LENGTH, 4)
    # Predict humidity
    print("Predicting humidity...")
    future_predictions_humi = best_model_humi.predict(X[-SEQ_LENGTH:])
    future_predictions_humi_copies = np.repeat(future_predictions_humi, 7, axis=-1)
    future_predicted_humi = scaler.inverse_transform(future_predictions_humi_copies)[:,4]
    return future_predicted_humi

In [67]:
predict_humi(14)

Predicting humidity...
1/1 [==============================] - 2s 2s/step


array([68.70022 , 63.505283, 68.79661 , 65.11355 , 70.81103 , 70.32317 ,
       66.195755, 75.05157 , 70.23435 , 70.18981 , 75.916855, 61.744247,
       62.372227, 64.294334], dtype=float32)

In [38]:
def predict_cloud(SEQ_LENGTH, scaled_data=scaled_data, best_model_cloud=best_model_cloud):
    # Create sequences for the entire dataset
    X, y = create_sequences_custom(scaled_data, SEQ_LENGTH, 5)
    # Predict cloud
    print("Predicting cloud...")
    future_predictions_cloud = best_model_cloud.predict(X[-SEQ_LENGTH:])
    future_predictions_cloud_copies = np.repeat(future_predictions_cloud, 7, axis=-1)
    future_predicted_cloud = scaler.inverse_transform(future_predictions_cloud_copies)[:,5]
    return future_predicted_cloud

In [68]:
predict_cloud(14)

Predicting cloud...
1/1 [==============================] - 4s 4s/step


array([39.276012, 21.25648 , 45.477196, 28.440165, 33.970108, 23.98989 ,
       18.71803 , 47.99646 , 32.7933  , 41.93625 , 52.72061 , 27.313227,
       29.400568, 32.233456], dtype=float32)

In [39]:
def predict_pressure(SEQ_LENGTH, scaled_data=scaled_data, best_model_pressure=best_model_pressure):
    # Create sequences for the entire dataset
    X, y = create_sequences_custom(scaled_data, SEQ_LENGTH, 6)
    # Predict pressure
    print("Predicting pressure...")
    future_predictions_pressure = best_model_pressure.predict(X[-SEQ_LENGTH:])
    future_predictions_pressure_copies = np.repeat(future_predictions_pressure, 7, axis=-1)
    future_predicted_pressure = scaler.inverse_transform(future_predictions_pressure_copies)[:,6]
    return future_predicted_pressure

In [69]:
predict_pressure(14)

Predicting pressure...
1/1 [==============================] - 3s 3s/step


array([1011.9366 , 1012.512  , 1010.04443, 1013.8941 , 1010.9541 ,
       1009.56323, 1009.5672 , 1010.9689 , 1012.11145, 1011.8246 ,
       1010.3428 , 1014.1818 , 1011.53766, 1011.1886 ], dtype=float32)

In [40]:
from datetime import datetime, timedelta

# Get weather data at specific date
def get_specific_date(date):
  specific_date = pd.to_datetime(date)
  last_date = data.index[-1]
  last_date = pd.to_datetime(last_date)
  # if date is already listed, return that date's weather data
  if specific_date <= last_date:
    return data.loc[data.index[-1]]
  else:
    return forecast(specific_date)

In [58]:
# Forecast
from datetime import datetime, timedelta

def forecast(end_date):
  print("Generating dates...")
  last_date = data.index[-1]
  last_date = pd.to_datetime(last_date)
  last_date = last_date + timedelta(days=1)
  last_date = last_date.strftime('%-m/%-d/%Y')
  future_dates = pd.date_range(start=last_date, end=end_date, freq='D')
  print(f'future_dates: {future_dates}')

  n_steps = (pd.Timestamp(end_date) - pd.Timestamp(last_date)).days + 1
  print(f'n_steps: {n_steps}')

  future_predicted_temp = predict_max_temp(n_steps)
  future_predicted_min_temp = predict_min_temp(n_steps)
  future_predicted_wind = predict_wind(n_steps)
  future_predicted_rain = predict_rain(n_steps)
  future_predicted_humi = predict_humi(n_steps)
  future_predicted_cloud = predict_cloud(n_steps)
  future_predicted_pressure = predict_pressure(n_steps)

  # Create a DataFrame for the predicted temperatures and future dates
  predicted_df = pd.DataFrame({'Date': future_dates,
                               'max': future_predicted_temp.flatten(),
                               'min': future_predicted_min_temp.flatten(),
                               'wind': future_predicted_wind.flatten(),
                               'rain': future_predicted_rain.flatten(),
                                'humidi': future_predicted_humi.flatten(),
                               'cloud': future_predicted_cloud.flatten(),
                               'pressure': future_predicted_pressure.flatten()})
  # Set 'Date' as the index
  predicted_df.set_index('Date', inplace=True)
  # Append the predicted temperatures to the existing data
  data_predicted = pd.concat([data, predicted_df])
  # Reset the index
  data_predicted.index = pd.to_datetime(data_predicted.index)
  # Sort the data by date
  data_predicted = data_predicted.sort_index()
  data_predicted = data_predicted.rename_axis('date')
  # Fix incorrect data
  data_predicted.loc[data_predicted['rain'] < 0, 'rain'] = 0.0
  return data_predicted

# **Testing Ground**

In [72]:
test = get_specific_date('2020-01-01')
print(test)

max           31.0
min           25.0
wind           7.0
rain           2.9
humidi        71.0
cloud         41.0
pressure    1010.0
Name: 12/19/2020, dtype: float64


In [74]:
test2 = get_specific_date('2021-01-02')
print(test2)

Generating dates...
future_dates: DatetimeIndex(['2020-12-20', '2020-12-21', '2020-12-22', '2020-12-23',
               '2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27',
               '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31',
               '2021-01-01', '2021-01-02'],
              dtype='datetime64[ns]', freq='D')
n_steps: 14
Predicting max temperature...
1/1 [==============================] - 0s 53ms/step
Predicting min temperature...
1/1 [==============================] - 0s 54ms/step
Predicting wind speed...
1/1 [==============================] - 0s 55ms/step
Predicting rain...
1/1 [==============================] - 0s 132ms/step
Predicting humidity...
1/1 [==============================] - 0s 33ms/step
Predicting cloud...
1/1 [==============================] - 0s 29ms/step
Predicting pressure...
1/1 [==============================] - 0s 28ms/step
             max   min  wind  rain  humidi  cloud  pressure
date                                                  

In [76]:
test2.iloc[-1]

max           33.0
min           27.0
wind           9.0
rain           2.9
humidi        71.0
cloud         51.0
pressure    1009.0
Name: 2021-06-18 00:00:00, dtype: float64

In [19]:
print(data)

             max   min  wind  rain  humidi  cloud  pressure
date                                                       
1/1/2009    29.0  22.0   6.0  18.5    89.0   68.0    1010.0
1/1/2010    34.0  24.0   5.0   0.0    59.0   20.0    1010.0
1/1/2011    29.0  23.0   4.0   0.2    81.0   42.0    1008.0
1/1/2012    32.0  23.0   7.0   1.8    76.0   35.0    1012.0
1/1/2013    33.0  24.0   4.0   0.0    69.0   15.0    1010.0
...          ...   ...   ...   ...     ...    ...       ...
12/19/2016  30.0  23.0   6.0   2.5    84.0   64.0    1011.0
12/19/2017  30.0  21.0   9.0   0.0    59.0   50.0    1014.0
12/19/2018  34.0  25.0   5.0   0.0    67.0   28.0    1013.0
12/19/2019  33.0  25.0   9.0   0.0    66.0   38.0    1012.0
12/19/2020  31.0  25.0   7.0   2.9    71.0   41.0    1010.0

[4549 rows x 7 columns]


In [77]:
print(type(test))
print(type(test2))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [78]:
if type(test) == pd.core.series.Series:
  print("test is a Series")
else:
  print("test is not a Series")

if type(test2) == pd.core.frame.DataFrame:
  print("test2 is a DataFrame")
else:
  print("test2 is not a DataFrame")

test is a Series
test2 is a DataFrame


In [79]:
data.to_csv('weather2.csv')

In [81]:
new_data = pd.read_csv('weather2.csv')
print(new_data.index)

RangeIndex(start=0, stop=4549, step=1)


# **Update Data**

In [106]:
def update_data(forecast_output):
  if type(forecast_output) == pd.core.series.Series:
    return False
  elif type(forecast_output) == pd.core.frame.DataFrame:
    forecast_output.to_csv('weather.csv')
    return True

# **FWI Fomulate**

In [82]:
class InvalidLatitude(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value) + " is not a valid Latitude."

In [83]:
def ffmcCalculator(RH, TEMP, WIND, FFMCPrev, RAIN=0):
    Mo = 147.2 * (101 - FFMCPrev) / (59.5 + FFMCPrev)  # /* 1 */

    if RAIN > 0.5:
        rf = RAIN - 0.5  # /* 2 */

        if Mo <= 150:
            Mr = Mo + 42.5 * rf * (math.exp(-100 / (251 - Mo))) * (1 - math.exp(-6.93 / rf))  # /* 3a */
        else:
            Mr = Mo + 42.5 * rf * (math.exp(-100 / (251 - Mo))) * (1 - math.exp(-6.93 / rf)) + \
                 0.0015 * (Mo - 150) ** 2 * rf ** 0.5  # /* 3b */

        if Mr > 250:
            Mr = 250
        Mo = Mr

    Ed = 0.942 * RH ** 0.679 + 11 * math.exp((RH - 100) / 10) + 0.18 * (21.1 - TEMP) * (
                1 - math.exp(-0.115 * RH))  # /* 4 */

    if Mo > Ed:
        ko = 0.424 * (1 - (RH / 100) ** 1.7) + 0.0694 * WIND ** 0.5 * (1 - (RH / 100) ** 8)  # /* 6a */
        kd = ko * 0.581 * math.exp(0.0365 * TEMP)  # /* 6b */
        m = Ed + (Mo - Ed) * 10 ** (-kd)  # /* 8 */
    else:
        Ew = 0.618 * RH ** 0.753 + 10 * math.exp((RH - 100) / 10) + 0.18 * (21.1 - TEMP) * (
                    1 - math.exp(-0.115 * RH))  # /* 5 */

        if Mo < Ew:
            kl = 0.424 * (1 - ((100 - RH) / 100) ** 1.7) + 0.0694 * WIND ** 0.5 * (
                        1 - ((100 - RH) / 100) ** 8)  # /* 7a */
            kw = kl * 0.581 * math.exp(0.0365 * TEMP)  # /* 7b */
            m = Ew - (Ew - Mo) * 10 ** (-kw)  # /* 9 */
        else:
            m = Mo

    return 59.5 * (250 - m) / (147.2 + m)  # /* 10 */

In [84]:
def dmcCalculator(TEMP, RH, RAIN, DMCPrev, month=datetime.now().month):
    if RAIN > 1.5:
        re = 0.92 * RAIN - 1.27
        Mo = 20 + math.exp(5.6348 - DMCPrev / 43.43)
        if DMCPrev <= 33:
            B = 100 / (0.5 + 0.3 * DMCPrev)
        elif DMCPrev <= 65:
            B = 14 - 1.3 * (math.log(DMCPrev))
        else:
            B = 6.2 * math.log(DMCPrev) - 17.2
        Mr = Mo + 1000 * re / (48.77 + B * re)
        Pr = 244.72 - 43.43 * math.log(Mr - 20)
        if Pr > 0:
            DMCPrev = Pr
        else:
            DMCPrev = 0

    if TEMP > -1.1:
        Dl = DayLength(month)
        K = 1.894 * (TEMP + 1.1) * (100 - RH) * Dl * 0.000001
    else:
        K = 0

    DMC = DMCPrev + 100 * K
    return DMC

In [85]:
def DayLength(MONTH, Latitude=10.762622):
    DayLength46N = [6.5, 7.5, 9.0, 12.8, 13.9, 13.9, 12.4, 10.9, 9.4, 8.0, 7.0, 6.0]
    DayLength20N = [7.9, 8.4, 8.9, 9.5, 9.9, 10.2, 10.1, 9.7, 9.1, 8.6, 8.1, 7.8]
    DayLength20S = [10.1, 9.6, 9.1, 8.5, 8.1, 7.8, 7.9, 8.3, 8.9, 9.4, 9.9, 10.2]
    DayLength40S = [11.5, 10.5, 9.2, 7.9, 6.8, 6.2, 6.5, 7.4, 8.7, 10.0, 11.2, 11.8]

    retVal = None

    if Latitude <= 90 and Latitude > 33:
        retVal = DayLength46N[MONTH - 1]
    elif Latitude <= 33 and Latitude > 0.0:
        retVal = DayLength20N[MONTH - 1]
    elif Latitude <= 0.0 and Latitude > -30.0:
        retVal = DayLength20S[MONTH - 1]
    elif Latitude <= -30.0 and Latitude >= -90.0:
        retVal = DayLength40S[MONTH - 1]

    if retVal == None:
        raise InvalidLatitude(Latitude)

    return retVal

In [86]:
def DryingFactor(Month, Latitude=10.762622):
    LfN = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4, -1.6, -1.6]
    LfS = [6.4, 5.0, 2.4, 0.4, -1.6, -1.6, -1.6, -1.6, -1.6, 0.9, 3.8, 5.8]

    if Latitude > 0:
        retVal = LfN[Month]
    elif Latitude <= 0.0:
        retVal = LfS[Month]

    return retVal

In [87]:
def dcCalculator(TEMP, RAIN, DCPrev, month=datetime.now().month):
    if RAIN > 2.8:
        rd = 0.83 * RAIN - 1.27
        Qo = 800 * math.exp(-DCPrev / 400)
        Qr = Qo + 3.937 * rd
        Dr = 400 * math.log(800 / Qr)
        if Dr > 0:
            DCPrev = Dr
        else:
            DCPrev = 0

    Lf = DryingFactor(month - 1)
    if TEMP > -2.8:
        V = 0.36 * (TEMP + 2.8) + Lf
    else:
        V = Lf

    if V < 0:
        V = 0

    D = DCPrev + 0.5 * V
    return D

In [88]:
def ISIcalculator(windSpd, FFMC):
    fWIND = math.exp(0.05039 * windSpd)
    m = 147.2 * (101 - FFMC) / (59.5 + FFMC)
    fF = 91.9 * math.exp(-0.1386 * m) * (1 + (m) ** 5.31 / 49300000)
    ISI = 0.208 * fWIND * fF

    return ISI

In [89]:
def BUIcalculator(DMC, DC):
    if (DMC <= 0.4 * DC):
        BUI = 0.8 * DMC * DC / (DMC + 0.4 * DC)
    else:
        BUI = DMC - (1 - 0.8 * DC / (DMC + 0.4 * DC)) * (0.92 + (0.0114 * DMC) ** 1.7)

    return BUI

In [90]:
def FWIcalculator(ISI, BUI):
    if BUI <= 80:
        fD = 0.626 * (BUI) ** 0.809 + 2
    else:
        fD = 1000 / (25 + 108.64 * math.exp(-0.023 * BUI))

    B = 0.1 * ISI * fD

    if B > 1:
        S = math.exp(2.72 * (0.434 * math.log(B)) ** 0.647)
    else:
        S = B

    return S

In [91]:
def get_wind_direction(wind_degree):
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    index = round(wind_degree / 22.5)
    if(index >= 16):
        index = index % 16
    return directions[index]

In [92]:
def FWItrans(FWI):
    if (FWI <= 3.0):
        danger = "Low"
    elif (FWI > 3.0 and FWI <= 10.0):
        danger = "Medium"
    elif (FWI > 10.0 and FWI <= 23.0):
        danger = "High"
    else:
        danger = "Extream"
    return danger

In [98]:
import queue
import datetime
import math

def FWICalculate(humidity,
                 temperature,
                 windSpd,
                 rain):
    FFMCQueue = queue.Queue()
    dmcQueue = queue.Queue()
    dcQueue = queue.Queue()

    FFMCQueue.put(85)
    curFFMC = ffmcCalculator(humidity, temperature, windSpd,
                             FFMCQueue.get(), rain)
    print("FFMC: ", round(curFFMC, 3))
    FFMCQueue.put(curFFMC)

    if dmcQueue.empty():
        dmcQueue.put(6)
    curDMC = dmcCalculator(temperature, humidity,
                           rain, dmcQueue.get())
    print("DMC: ", round(curDMC, 3))
    dmcQueue.put(curDMC)

    if dcQueue.empty():
        dcQueue.put(15)
    curDC = dcCalculator(temperature, rain, dcQueue.get())
    print("DC: ", round(curDC, 3))
    dcQueue.put(curDC)

    curISI = ISIcalculator(windSpd, curFFMC)
    print("ISI: ", round(curISI, 3))

    curBUI = BUIcalculator(curDMC, curDC)
    print("BUI: ", round(curBUI, 3))

    curFWI = FWIcalculator(curISI, curBUI)
    print("FWI: ", round(curFWI, 3))

    print("Chance of fire is:", FWItrans(curFWI))

# **Using API to fetch weather condition**

In [95]:
import requests
import datetime

# API Key
LAT = 10.762622
LON = 106.660172
API_KEY = '6c854c10dc3e46c6a1e113713241004'

In [96]:
import pytz
from datetime import datetime
import requests

def get_real_weather_data(input_date=datetime.now(), api_key=API_KEY,lat=LAT,lon=LON):
  # Extract relevant data from the weather_data object
  extracted_data = []
  current_date = input_date
  # Format the current date in the required format (YYYY-MM-DD)
  date = current_date.strftime("%Y-%m-%d")
  # API endpoint URL for the current date
  url = f'https://api.weatherapi.com/v1/history.json?key={api_key}&q={lat},{lon}&dt={date}'
  # Make the API request
  response = requests.get(url)
  # Define the UTC+7 timezone
  utc_7 = pytz.timezone('Asia/Bangkok')
  # Get the current datetime in UTC
  utc_now = datetime.now(pytz.utc)
  # Convert UTC datetime to UTC+7
  utc_7_now = utc_now.astimezone(utc_7).hour
  # Check if the request was successful
  if response.status_code == 200:
    # Extract the weather data from the response
    weather_data = response.json()
    maxtemp = weather_data['forecast']['forecastday'][0]['hour'][12]['temp_c']
    mintemp = weather_data['forecast']['forecastday'][0]['hour'][23]['temp_c']
    wind = weather_data['forecast']['forecastday'][0]['hour'][12]['wind_kph']
    wind_d = weather_data['forecast']['forecastday'][0]['hour'][utc_7_now]['wind_dir']
    rain = weather_data['forecast']['forecastday'][0]['day']['totalprecip_mm']
    humidi = weather_data['forecast']['forecastday'][0]['hour'][12]['humidity']
    cloud = weather_data['forecast']['forecastday'][0]['hour'][utc_7_now]['cloud']
    pressure = weather_data['forecast']['forecastday'][0]['hour'][utc_7_now]['pressure_mb']
    extracted_data.append([maxtemp, mintemp, wind, wind_d, rain, humidi, cloud, pressure, date])
    return extracted_data
  else:
    print('Error occurred while fetching weather data.')
    return None

In [97]:
def print_weather_data(extracted_data):
  maxtemp = extracted_data[0][0]
  mintemp = extracted_data[0][1]
  wind = extracted_data[0][2]
  wind_d = extracted_data[0][3]
  rain = extracted_data[0][4]
  humidi = extracted_data[0][5]
  cloud = extracted_data[0][6]
  pressure = extracted_data[0][7]
  date = extracted_data[0][8]
  # Print or process the weather data as needed
  print(f'Date: {date}')
  print(f"Max Temperature: {maxtemp} °C")
  print(f"Min Temperature: {mintemp} °C")
  print(f"Wind Speed: {wind} kph")
  print(f"Wind Direction: {wind_d}")
  print(f"Rain: {rain} mm")
  print(f"Humidity: {humidi} %")
  print(f"Cloud: {cloud} %")
  print(f"Pressure: {pressure} mb")

# **Get FWI**

In [110]:
from datetime import datetime

def get_fwi(choice=0):
  if(choice == 0):
    today = get_real_weather_data()
    return FWICalculate(today[0][5], today[0][0], today[0][2], today[0][4])
  elif(choice == 1):
    input_day = input("Enter the day you want to check:")
    input_month = input("Enter the month you want to check:")
    input_year = input("Enter the year you want to check:")
    input_date = input_year + '-' + input_month + '-' + input_day

    input_date_weather = get_specific_date(input_date)
    if(update_data(input_date_weather) == True):
      input_date_weather = input_date_weather.iloc[-1]
      global data
      data = pd.read_csv('weather.csv')
      data.index = data.date
      data = data.drop('date', axis=1)
      print("Data table updated!")
      print(data)

    humi = input_date_weather['humidi'].item()
    max = input_date_weather['max'].item()
    wind = input_date_weather['wind'].item()
    rain = input_date_weather['rain'].item()

    return FWICalculate(humi, max, wind, rain)

In [104]:
get_fwi()

FFMC:  91.292
DMC:  9.945
DC:  22.56
ISI:  10.282
BUI:  9.899
FWI:  10.325
Chance of fire is: High


In [111]:
get_fwi(1)

Enter the day you want to check:2
Enter the month you want to check:5
Enter the year you want to check:2024
Generating dates...
future_dates: DatetimeIndex(['2020-12-20', '2020-12-21', '2020-12-22', '2020-12-23',
               '2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27',
               '2020-12-28', '2020-12-29',
               ...
               '2024-04-23', '2024-04-24', '2024-04-25', '2024-04-26',
               '2024-04-27', '2024-04-28', '2024-04-29', '2024-04-30',
               '2024-05-01', '2024-05-02'],
              dtype='datetime64[ns]', length=1230, freq='D')
n_steps: 1230
Predicting max temperature...
39/39 [==============================] - 32s 818ms/step
Predicting min temperature...
39/39 [==============================] - 31s 796ms/step
Predicting wind speed...
39/39 [==============================] - 31s 796ms/step
Predicting rain...
39/39 [==============================] - 33s 834ms/step
Predicting humidity...
39/39 [==============================] - 3

In [114]:
get_fwi(1)

Enter the day you want to check:2
Enter the month you want to check:5
Enter the year you want to check:2024
FFMC:  87.245
DMC:  8.356
DC:  22.008
ISI:  6.348
BUI:  8.574
FWI:  6.308
Chance of fire is: Medium
